In [1]:
import torch
import h5py
import os
import sys
direc = "/mnt/data/shared/jacob/GFNO"
os.chdir(direc)
sys.path.append(direc)

from models.FNO import FNO2d, FNO3d
from models.GFNO import GFNO2d, GFNO3d
from models.Unet import Unet_Rot_M, Unet_Rot_3D
from models.radialNO import radialNO2d, radialNO3d
from utils import pde_data
import numpy as np
import scipy
import timeit
import pandas as pd
from tqdm import tqdm
import GPUtil

GPU = "7"
os.environ["CUDA_VISIBLE_DEVICES"] = GPU

In [2]:
path = "./data/"

In [3]:
TRAIN_PATH = path + "ns_V1e-4_N10000_T30.mat"
try:
    with h5py.File(TRAIN_PATH, 'r') as f:
        data = np.array(f['u'])
    data = np.transpose(data, axes=range(len(data.shape) - 1, -1, -1))
except:
    data = scipy.io.loadmat(os.path.expandvars(TRAIN_PATH))['u'].astype(np.float32)

data = data[..., None] # add channel dim

data = torch.from_numpy(data)

In [4]:
ntest = 100

T_in = 10
T = 20

batch_size2d = 20
batch_size3d = 10

strategy2d = "teacher_forcing"
strategy3d = "oneshot"

test = data[-ntest:]

test_data2d = pde_data(test, train=False, strategy=strategy2d, T_in=T_in, T_out=T)
test_data3d = pde_data(test, train=False, strategy=strategy3d, T_in=T_in, T_out=T)

test_loader2d = torch.utils.data.DataLoader(test_data2d, batch_size=batch_size2d, shuffle=False)
test_loader3d = torch.utils.data.DataLoader(test_data3d, batch_size=batch_size3d, shuffle=False)

In [5]:
channels = {"FNO2d":20,
"GFNO2d_p4":10,
"GFNO2d_p4m":7,
"radialNO2d_p4":40,
"radialNO2d_p4m":50,
"Unet_Rot_M2d":32,
"FNO3d":20,
"GFNO3d_p4":11,
"GFNO3d_p4m":7,
"radialNO3d_p4":60,
"radialNO3d_p4m":80,
"Unet_Rot_3D":32}
inference_times = {}

In [6]:
def inference_time(model, loader, n=10000, warmup=100, return_times=False):

    torch.cuda.empty_cache()

    model.cuda()
    model.eval()
    params = sum(par.numel() * (1 + par.is_complex()) for par in model.parameters())
    times = []
    with torch.no_grad():
        for trial in tqdm(range((n + warmup) // len(loader) + 1)):
            for x, _ in loader:
                x = x.cuda()
                start = timeit.default_timer()
                model(x)
                elapsed = timeit.default_timer() - start
                if trial > warmup:
                    times.append(elapsed)
    torch.cuda.synchronize()

    memory = GPUtil.getGPUs()[int(GPU)].memoryUsed

    times = times[:n]
    if return_times:
        return times
    mean = torch.tensor(times).mean()
    std = torch.tensor(times).std()

    return {"params": params, "mean":mean.item(), "std":std.item(), "memory":memory}

In [8]:
# FNO2d
model_name = 'FNO2d'
model = FNO2d(num_channels=1, initial_step=T_in, modes1=12, modes2=12, width=channels[model_name], grid_type='cartesian').cuda()
inference_times[model_name] = inference_time(model, test_loader2d)
inference_times[model_name]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2021/2021 [02:04<00:00, 16.23it/s]


{'params': 928661,
 'mean': 0.004265904426574707,
 'std': 0.0016596349887549877,
 'memory': 1869.0}

In [9]:
# GFNO2d_p4
model_name = 'GFNO2d_p4'
model = GFNO2d(num_channels=1, initial_step=T_in, modes=12, width=channels[model_name],
               reflection=False, grid_type="symmetric").cuda()
inference_times[model_name] = inference_time(model, test_loader2d)
inference_times[model_name]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2021/2021 [02:00<00:00, 16.77it/s]


{'params': 853121,
 'mean': 0.0044963667169213295,
 'std': 0.0021232448052614927,
 'memory': 2019.0}

In [10]:
# GFNO2d_p4m
model_name = 'GFNO2d_p4m'
model = GFNO2d(num_channels=1, initial_step=T_in, modes=12, width=channels[model_name],
               reflection=True, grid_type="symmetric").cuda()
inference_times[model_name] = inference_time(model, test_loader2d)
inference_times[model_name]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2021/2021 [02:17<00:00, 14.66it/s]


{'params': 835969,
 'mean': 0.004812923260033131,
 'std': 0.0037937520537525415,
 'memory': 2223.0}

In [11]:
# radialNO2d p4
model_name = 'radialNO2d_p4'
model = radialNO2d(num_channels=1, initial_step=T_in, modes=12, width=channels[model_name], reflection=False,
                       grid_type="symmetric").cuda()
inference_times[model_name] = inference_time(model, test_loader2d)
inference_times[model_name]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2021/2021 [02:01<00:00, 16.66it/s]


{'params': 1031681,
 'mean': 0.004344562068581581,
 'std': 0.002626420697197318,
 'memory': 2013.0}

In [12]:
# radialNO2d p4m
model_name = 'radialNO2d_p4m'
model = radialNO2d(num_channels=1, initial_step=T_in, modes=12, width=channels[model_name], reflection=True,
                       grid_type="symmetric").cuda()
inference_times[model_name] = inference_time(model, test_loader2d)
inference_times[model_name]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2021/2021 [02:10<00:00, 15.45it/s]


{'params': 951601,
 'mean': 0.004159676376730204,
 'std': 0.003289111191406846,
 'memory': 2081.0}

In [13]:
# Unet_Rot_M
model_name = 'Unet_Rot_M2d'
model = Unet_Rot_M(input_frames=T_in * 1, output_frames=1, kernel_size=3, N=4, grid_type="symmetric", width=channels[model_name]).cuda()
inference_times[model_name] = inference_time(model, test_loader2d)
inference_times[model_name]

/data/jacob/anaconda3/envs/GFNO/lib/python3.8/site-packages/e2cnn/nn/modules/r2_conv/basisexpansion_singleblock.py:80: UserWarning: indexing with dtype torch.uint8 is now deprecated, please use a dtype torch.bool instead. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525552411/work/aten/src/ATen/native/IndexingUtils.h:27.)
  full_mask[mask] = norms.to(torch.uint8)
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2021/2021 [03:00<00:00, 11.17it/s]


{'params': 3651719,
 'mean': 0.007919750176370144,
 'std': 0.004336921032518148,
 'memory': 2361.0}

In [14]:
# FNO3d
model_name = 'FNO3d'
model = FNO3d(num_channels=1, initial_step=T_in, modes1=8, modes2=8, modes3=6, width=channels[model_name], time=True, time_pad=False).cuda()
inference_times[model_name] = inference_time(model, test_loader3d)
inference_times[model_name]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1011/1011 [04:03<00:00,  4.15it/s]


{'params': 4922281,
 'mean': 0.0087118623778224,
 'std': 0.0075395251624286175,
 'memory': 3163.0}

In [15]:
# GFNO3d_p4
model_name = 'GFNO3d_p4'
model = GFNO3d(num_channels=1, initial_step=T_in, modes=8, time_modes=6, width=channels[model_name], reflection=False, grid_type="symmetric", time_pad=False).cuda()
inference_times[model_name] = inference_time(model, test_loader3d)
inference_times[model_name]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1011/1011 [06:36<00:00,  2.55it/s]


{'params': 4799708,
 'mean': 0.010207905434072018,
 'std': 0.01561953779309988,
 'memory': 4589.0}

In [16]:
# GFNO3d_p4m
model_name = 'GFNO3d_p4m'
model = GFNO3d(num_channels=1, initial_step=T_in, modes=8, time_modes=6, width=channels[model_name], reflection=True, grid_type="symmetric", time_pad=False).cuda()
inference_times[model_name] = inference_time(model, test_loader3d)
inference_times[model_name]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1011/1011 [07:39<00:00,  2.20it/s]


{'params': 3887304,
 'mean': 0.010730269365012646,
 'std': 0.016459759324789047,
 'memory': 5535.0}

In [17]:
# radialNO3d p4
model_name = 'radialNO3d_p4'
model = radialNO3d(num_channels=1, initial_step=T_in, modes=8, time_modes=6, width=channels[model_name], reflection=False, grid_type="symmetric", time_pad=False).cuda()
inference_times[model_name] = inference_time(model, test_loader3d)
inference_times[model_name]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1011/1011 [08:01<00:00,  2.10it/s]


{'params': 4984381,
 'mean': 0.010807642713189125,
 'std': 0.016583405435085297,
 'memory': 6391.0}

In [18]:
# radialNO3d p4m
model_name = 'radialNO3d_p4m'
model = radialNO3d(num_channels=1, initial_step=T_in, modes=8, time_modes=6, width=channels[model_name], reflection=True, grid_type="symmetric", time_pad=False).cuda()
inference_times[model_name] = inference_time(model, test_loader3d)
inference_times[model_name]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1011/1011 [09:36<00:00,  1.75it/s]


{'params': 5634641,
 'mean': 0.011534293182194233,
 'std': 0.017982367426156998,
 'memory': 7683.0}

In [19]:
# Unet_Rot_3d
model_name = 'Unet_Rot_3D'
model = Unet_Rot_3D(input_frames=T_in * 1, output_frames=1, kernel_size=3, N=4, grid_type='symmetric', width=channels[model_name]).cuda()
inference_times[model_name] = inference_time(model, test_loader3d)
inference_times[model_name]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1011/1011 [13:49<00:00,  1.22it/s]


{'params': 6084481,
 'mean': 0.020287223160266876,
 'std': 0.01576860062777996,
 'memory': 7631.0}

In [20]:
inference_times.keys()

dict_keys(['FNO2d', 'GFNO2d_p4', 'GFNO2d_p4m', 'radialNO2d_p4', 'radialNO2d_p4m', 'Unet_Rot_M2d', 'FNO3d', 'GFNO3d_p4', 'GFNO3d_p4m', 'radialNO3d_p4', 'radialNO3d_p4m', 'Unet_Rot_3D'])

In [21]:
precision = 2
model_summaries = {model:{"\\# Parameters (M)":round(stats['params'] / 1e6, precision),
                          "Inference Time (ms)":f"{round(stats['mean'] * 1000, precision)}({round(stats['std'] * 1000, precision)})",
                          "Forward Memory (GiB)":round(stats['memory'] / 1024, precision)} for model, stats in inference_times.items()}

In [29]:
model_summaries

{'FNO2d': {'\\# Parameters (M)': 0.93,
  'Inference Time (ms)': '4.27(1.66)',
  'Forward Memory (GiB)': 1.83},
 'GFNO2d_p4': {'\\# Parameters (M)': 0.85,
  'Inference Time (ms)': '4.5(2.12)',
  'Forward Memory (GiB)': 1.97},
 'GFNO2d_p4m': {'\\# Parameters (M)': 0.84,
  'Inference Time (ms)': '4.81(3.79)',
  'Forward Memory (GiB)': 2.17},
 'radialNO2d_p4': {'\\# Parameters (M)': 1.03,
  'Inference Time (ms)': '4.34(2.63)',
  'Forward Memory (GiB)': 1.97},
 'radialNO2d_p4m': {'\\# Parameters (M)': 0.95,
  'Inference Time (ms)': '4.16(3.29)',
  'Forward Memory (GiB)': 2.03},
 'Unet_Rot_M2d': {'\\# Parameters (M)': 3.65,
  'Inference Time (ms)': '7.92(4.34)',
  'Forward Memory (GiB)': 2.31},
 'FNO3d': {'\\# Parameters (M)': 4.92,
  'Inference Time (ms)': '8.71(7.54)',
  'Forward Memory (GiB)': 3.09},
 'GFNO3d_p4': {'\\# Parameters (M)': 4.8,
  'Inference Time (ms)': '10.21(15.62)',
  'Forward Memory (GiB)': 4.48},
 'GFNO3d_p4m': {'\\# Parameters (M)': 3.89,
  'Inference Time (ms)': '10.73

In [22]:
stat_names = model_summaries['FNO2d'].keys()

In [23]:
model_names = {'FNO':{'2D':'FNO2d', '3D':'FNO3d'}, '$G$-FNO-$p4$':{'2D':'GFNO2d_p4', '3D':'GFNO3d_p4'}, '$G$-FNO-$p4m$':{'2D':'GFNO2d_p4m', '3D':'GFNO3d_p4m'}, 'radialFNO-$p4$': {'2D':'radialNO2d_p4', '3D':'radialNO3d_p4'}, 'radialFNO-$p4m$': {'2D':'radialNO2d_p4m', '3D':'radialNO3d_p4m'}, 'U-Net-$p4$': {'2D':'Unet_Rot_M2d', '3D':'Unet_Rot_3D'}}

In [24]:
model_stats = {(dimension, stat_name): {model:model_summaries[model_names[model][dimension]][stat_name] for model in model_names.keys()} for dimension in {'3D', '2D'} for stat_name in stat_names}

In [25]:
model_stats['2D', '\# Parameters (M)']

{'FNO': 0.93,
 '$G$-FNO-$p4$': 0.85,
 '$G$-FNO-$p4m$': 0.84,
 'radialFNO-$p4$': 1.03,
 'radialFNO-$p4m$': 0.95,
 'U-Net-$p4$': 3.65}

In [26]:
df = pd.DataFrame(model_stats)
df = df[['2D', '3D']]

In [27]:
df

2D                                           \
                \# Parameters (M) Inference Time (ms) Forward Memory (GiB)   
FNO                          0.93          4.27(1.66)                 1.83   
$G$-FNO-$p4$                 0.85           4.5(2.12)                 1.97   
$G$-FNO-$p4m$                0.84          4.81(3.79)                 2.17   
radialFNO-$p4$               1.03          4.34(2.63)                 1.97   
radialFNO-$p4m$              0.95          4.16(3.29)                 2.03   
U-Net-$p4$                   3.65          7.92(4.34)                 2.31   

                               3D                                           
                \# Parameters (M) Inference Time (ms) Forward Memory (GiB)  
FNO                          4.92          8.71(7.54)                 3.09  
$G$-FNO-$p4$                 4.80        10.21(15.62)                 4.48  
$G$-FNO-$p4m$                3.89        10.73(16.46)                 5.41  
radialFNO-$p4$               4.98        10.81(16.58)                 6.24  
radialFNO-$p4m$              5.63        11.53(17.98)                 7.50  
U-Net-$p4$                   6.08        20.29(15.77)                 7.45

In [28]:
print(df.to_latex(escape=False))

\begin{tabular}{lrlrrlr}
\toprule
{} & \multicolumn{3}{l}{2D} & \multicolumn{3}{l}{3D} \\
{} & \# Parameters (M) & Inference Time (ms) & Forward Memory (GiB) & \# Parameters (M) & Inference Time (ms) & Forward Memory (GiB) \\
\midrule
FNO             &              0.93 &          4.27(1.66) &                 1.83 &              4.92 &          8.71(7.54) &                 3.09 \\
$G$-FNO-$p4$    &              0.85 &           4.5(2.12) &                 1.97 &              4.80 &        10.21(15.62) &                 4.48 \\
$G$-FNO-$p4m$   &              0.84 &          4.81(3.79) &                 2.17 &              3.89 &        10.73(16.46) &                 5.41 \\
radialFNO-$p4$  &              1.03 &          4.34(2.63) &                 1.97 &              4.98 &        10.81(16.58) &                 6.24 \\
radialFNO-$p4m$ &              0.95 &          4.16(3.29) &                 2.03 &              5.63 &        11.53(17.98) &                 7.50 \\
U-Net-$p4$      &   

/tmp/ipykernel_486331/3445587271.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex(escape=False))
